# Quantum Information Processing

Following: http://qutip.org/docs/latest/guide/guide-qip.html

## Quantum Circuit

In [1]:
from qutip.qip.circuit import QubitCircuit, Gate
from qutip.qip.operations import gate_sequence_product

In [2]:
qc = QubitCircuit(N=2)

# Gates are class objects inside `qutip.qip.operations.Gate`
swap_gate = Gate(name="SWAP", targets=[0, 1])
qc.add_gate(swap_gate)
qc.add_gate("CNOT", controls=0, targets=1)
qc.add_gate(swap_gate)
print(qc.gates)

[Gate(SWAP, targets=[0, 1], controls=None), Gate(CNOT, targets=[1], controls=[0]), Gate(SWAP, targets=[0, 1], controls=None)]


In [3]:
# Get the matrix representation with `qutip.qip.QubitCircuit.propagators`
U_list = qc.propagators()
print(len(U_list))
print("----")
print(U_list[0])
print("----")
print(gate_sequence_product(U_list))

3
----
Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]]
----
Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]]


In [4]:
qc_temp = QubitCircuit(N=2)
qc_temp.add_gate(swap_gate)
print(qc_temp.propagators())

[Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]]]


In [5]:
qc_temp = QubitCircuit(N=2)
qc_temp.add_gate("CNOT", controls=0, targets=1)
qc_temp.add_gate("CNOT", controls=1, targets=0)
qc_temp.add_gate("CNOT", controls=0, targets=1)
print(qc_temp.propagators())

[Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]], Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]], Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]]]


In [6]:
print(gate_sequence_product(qc_temp.propagators()))

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]]


There are lots of pre-defined gates - e.g., "RX", "SNOT" (Hadamard), "SQRTISWAP", etc. See the documentation.

We may also define our own gates:

In [7]:
from qutip.qip.operations import rx
from qutip import Qobj
import numpy as np

In [8]:
def user_gate1(arg_value):
    # controlled rotation X
    mat = np.zeros((4, 4), dtype=np.complex)
    mat[0, 0] = mat[1, 1] = 1.
    mat[2:4, 2:4] = rx(arg_value)
    return Qobj(mat, dims=[[2, 2], [2, 2]])

In [9]:
def user_gate2():
    # S gate
    mat = np.array([[1., 0.],
                    [0., 1.j]])
    return Qobj(mat, dims=[[2], [2]])

In [10]:
qc = QubitCircuit(2)
qc.user_gates = {"CTRLRX": user_gate1, "S": user_gate2}
qc.add_gate("CTRLRX", targets=[0, 1], arg_value=np.pi/2)
qc.add_gate("CTRLRX", targets=[1, 0], arg_value=np.pi/2)
g_T = Gate("S", targets=[1])
qc.add_gate(g_T)

In [11]:
props = qc.propagators()

In [12]:
props[0]

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[1.        +0.j         0.        +0.j         0.        +0.j
  0.        +0.j        ]
 [0.        +0.j         1.        +0.j         0.        +0.j
  0.        +0.j        ]
 [0.        +0.j         0.        +0.j         0.70710678+0.j
  0.        -0.70710678j]
 [0.        +0.j         0.        +0.j         0.        -0.70710678j
  0.70710678+0.j        ]]

In [13]:
len(props)

3

In [14]:
props[1]

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[1.        +0.j         0.        +0.j         0.        +0.j
  0.        +0.j        ]
 [0.        +0.j         0.70710678+0.j         0.        +0.j
  0.        -0.70710678j]
 [0.        +0.j         0.        +0.j         1.        +0.j
  0.        +0.j        ]
 [0.        +0.j         0.        -0.70710678j 0.        +0.j
  0.70710678+0.j        ]]

In [15]:
props[2]

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+1.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+1.j]]

In [16]:
props[2].dag() * props[2]

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]

In [17]:
gate_sequence_product(props)

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[1.        +0.j         0.        +0.j         0.        +0.j
  0.        +0.j        ]
 [0.        +0.j         0.        +0.70710678j 0.        -0.5j
  0.5       +0.j        ]
 [0.        +0.j         0.        +0.j         0.70710678+0.j
  0.        -0.70710678j]
 [0.        +0.j         0.70710678+0.j         0.5       +0.j
  0.        +0.5j       ]]

## Processor for QIP simulation

Based on the open system solver - the processor is a simulator of a quantum device on which the circuit will be implemented. The device is controlled with pulses.

In [18]:
from qutip.qip.device import Processor

In [19]:
proc = Processor(2)

In [20]:
from qutip.operators import sigmaz

In [21]:
proc.add_control(sigmaz(), cyclic_permutation=True)
proc.pulses[0].coeffs = np.array([[1.0, 1.5, 2.0],
                                  [1.8, 1.3, 0.8]])
proc.pulses[0].tlist = np.array([0.1, 0.2, 0.4, 0.5])

`qutip.qip.device.ModelProcessor` is more experiment-oriented and based on physical models. A universal set of gates is defined on the processor as well as the pulses implementing them in this specific physical model. There is another approach based on the optimal control module in QuTiP. There we define available Hamiltonians and the processor uses algorithms to find the optimal control pulses that realize the desired unitary.

### Spin Chain

The control Hamiltonians are $\sigma_x$, $\sigma_z$, and $\sigma_x \sigma_x + \sigma_y \sigma_y$. This processor will first decompose the gate into the the universal gate set with ISWAP and SQRTISWAP as two qubit gates, resolve them into quantum gates of adjacent qubits, and then compute the pulse coefficients.

### Dispersive cQED

A simulator based on Cavity QED. The control Hamiltonians are the single qubit-rotation together with the qubits-vacity interaction $a^{\dagger} \sigma^-  + a \sigma^+$.

### OptPulseProcessor

The Hamiltonian includes a drift part and a control part.

In [22]:
from qutip.qip.device import OptPulseProcessor
from qutip.operators import sigmaz, sigmax, sigmay
from qutip.tensor import tensor

In [23]:
# same parameter for all gates
qc = QubitCircuit(N=1)
qc.add_gate("SNOT", 0)

In [24]:
num_tslots = 10
evo_time = 10
processor = OptPulseProcessor(N=1, drift=sigmaz())
processor.add_control(sigmax())
tlist, coeffs = processor.load_circuit(
    qc, num_tslots=num_tslots, evo_time=evo_time
)

In [25]:
tlist

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.])

In [26]:
coeffs

array([[ 0.92953637,  0.83046649,  0.5186285 , -0.17935601,  0.14984946,
        -0.21224641,  0.62587635,  0.07777727,  0.49509852,  0.50411125]])

In [27]:
# different parameters for different gates
qc = QubitCircuit(N=2)
qc.add_gate('SNOT', 0)
qc.add_gate('SWAP', targets=[0, 1])
qc.add_gate('CNOT', controls=1, targets=[0])

In [28]:
processor = OptPulseProcessor(N=2, drift=tensor([sigmaz()]*2))
processor.add_control(sigmax(), cyclic_permutation=True)
processor.add_control(sigmay(), cyclic_permutation=True)
processor.add_control(tensor([sigmay(), sigmay()]))

In [29]:
setting_args = {
    'SNOT': {'num_tslots': 10, 'evo_time': 1},
    'SWAP': {'num_tslots': 30, 'evo_time': 3},
    'CNOT': {'num_tslots': 30, 'evo_time': 3}
}

In [30]:
tlist, coeffs = processor.load_circuit(
    qc, setting_args=setting_args, merge_gates=False)

In [31]:
tlist

array([0.        , 0.1       , 0.2       , 0.3       , 0.40000001,
       0.50000001, 0.60000001, 0.70000001, 0.80000001, 0.90000001,
       1.00000001, 1.10000002, 1.20000002, 1.30000002, 1.40000002,
       1.50000002, 1.60000002, 1.70000003, 1.80000003, 1.90000003,
       2.00000003, 2.10000003, 2.20000003, 2.30000003, 2.40000004,
       2.50000004, 2.60000004, 2.70000004, 2.80000004, 2.90000004,
       3.00000004, 3.10000005, 3.20000005, 3.30000005, 3.40000005,
       3.50000005, 3.60000005, 3.70000006, 3.80000006, 3.90000006,
       4.00000006, 4.10000006, 4.20000006, 4.30000006, 4.40000007,
       4.50000007, 4.60000007, 4.70000007, 4.80000007, 4.90000007,
       5.00000007, 5.10000008, 5.20000008, 5.30000008, 5.40000008,
       5.50000008, 5.60000008, 5.70000008, 5.80000009, 5.90000009,
       6.00000009, 6.10000009, 6.20000009, 6.30000009, 6.4000001 ,
       6.5000001 , 6.6000001 , 6.7000001 , 6.8000001 , 6.9000001 ,
       7.0000001 ])

In [32]:
coeffs

array([[ 2.76850764e-01, -4.99546127e+00, -1.37583306e+00,
        -3.30901729e-02,  8.29303721e-01, -2.41545384e+00,
        -2.15843613e+00, -2.83400578e+00, -1.38759898e+00,
        -9.87685251e-02, -5.55477799e-01, -7.45907761e-01,
         1.40659671e-01,  3.02580362e-01, -6.82625003e-01,
         4.97931893e-01,  9.14871375e-02, -4.38791122e-01,
         2.19752745e-01,  5.38107311e-01, -7.92338566e-01,
        -5.01630284e-01,  1.16991559e+00, -2.78891714e-01,
        -2.48318183e-01, -4.29666943e-01, -1.08708852e-02,
        -1.88151887e-01, -1.01965402e+00,  4.83182976e-02,
         7.31408675e-01, -2.46725532e-01,  4.83196146e-01,
        -8.79396278e-02,  4.60005708e-01, -8.27448000e-02,
        -4.11816988e-01,  2.39784283e-01, -4.46102065e-02,
         1.32697522e-01,  8.47661330e-02, -5.69401088e-02,
        -6.02001648e-01, -1.02120350e+00, -3.88135830e-01,
         7.18481057e-01, -5.45237677e-01, -7.18930115e-01,
         1.06197763e+00,  1.07291061e+00,  5.02833218e-0

## Noise Simulation

...